In [ ]:
import geemap
import ee
import pandas as pd
import geemap.colormaps as cm

from egis.utils import (
    vis_params_dict, extract_single_img
)

In [ ]:
ee.Initialize()

## Interactive usage

In [ ]:
# helper function for interactive use
def get_coordinates_list(geo_obj):
    return geo_obj.getInfo()['coordinates']

In [ ]:
Map = geemap.Map()
Map

In [ ]:
get_coordinates_list(Map.user_roi)

## Define selection

In [ ]:
coord_list = [[-101.060366, 37.882811],
  [-100.675393, 37.882811],
  [-100.675393, 38.034941],
  [-101.060366, 38.034941],
  [-101.060366, 37.882811]]
roi_clip_region = ee.Geometry.Polygon(coord_list)

In [ ]:
center_coords = [-100.897585, 37.973621]
center_coords_map = [center_coords[1], center_coords[0]]
center_point = ee.Geometry.Point(center_coords)

In [ ]:
date_beg = '2022-07-20'
date_end = '2022-08-20'

In [ ]:
dataset_sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                          .filterDate(date_beg, date_end) \
                          .filterBounds(center_point) \
                          .map(lambda img: img.clip(roi_clip_region))

## Inspect metadata of collection

In [ ]:
img_dates = geemap.image_dates(dataset_sentinel).getInfo()
img_dates

In [ ]:
relevant_props = ["system:id", "system:time_start", 'CLOUDY_PIXEL_OVER_LAND_PERCENTAGE', 
                  'CLOUDY_PIXEL_PERCENTAGE', 'CLOUD_SHADOW_PERCENTAGE', 'DARK_FEATURES_PERCENTAGE']

relevant_props = ["system:id", "system:time_start"]

In [ ]:
n_imgs = len(img_dates)

img_df_list = []

for ii in range(0, n_imgs):
    # get image
    this_img = extract_single_img(dataset_sentinel, idx=ii)
    
    # get image properties as dict
    this_img_prop_dict = geemap.image_props(this_img).getInfo()
    this_dict = {k: this_img_prop_dict[k] for k in relevant_props}
    
    # transform to DataFrame
    this_img_df = pd.DataFrame.from_dict({0: this_dict}, orient='index')
    img_df_list.append(this_img_df)
    
all_img_info_df = pd.concat(img_df_list, axis=0)
all_img_info_df = all_img_info_df.set_index("system:id")
all_img_info_df

## Visualize images

In [ ]:
vis_params = vis_params_dict(bands=['B4','B3','B2'],
                                     min=0,
                                     max=3000,
                                     gamma=1.4)

In [ ]:
# all_img_info_df.index[2]

In [ ]:
sentinel_map = geemap.Map()

for ii in range(0, n_imgs):
    # get image
    this_img = extract_single_img(dataset_sentinel, idx=ii)
    layer_name = f'img_{ii}'
    
    sentinel_map.addLayer(this_img, vis_params, layer_name)
    
sentinel_map.centerObject(center_point, 12)
sentinel_map

## Inspect image counts

In [ ]:
img_count = geemap.image_count(dataset_sentinel)

In [ ]:
vis_params = {'min': 0, 'max': 5, 'palette': cm.palettes.coolwarm}

In [ ]:
Map = geemap.Map(center=center_coords_map, zoom=8)
Map.addLayer(img_count, vis_params)
Map